# Metrics and Plots for the trained models

This notebook takes trained models objects found in their reosective paths and creats results tables in `results/scores_tables`

> beware, running it might override files if new models are added

In [138]:
seed=9

In [139]:
MAIN_DUMP=f'../../dump_seeds/dump_{seed}'
import os,sys
from joblib import Parallel, delayed,dump,load

sys.path.append(os.path.abspath('../../src/ml'))
from load_matrix import load_df
from model_trainer import MLModel
from utils import load_models

import pandas as pd

from evaluator import Evaluator
from collector import ResultsCollector
from visualizer import DashboardVisualizer


## v2.10

In [140]:
current_version='v2.10'
standard_models=load_models(MAIN_DUMP,version=current_version,normalization='standard')
robust_models=load_models(MAIN_DUMP,version=current_version,normalization='robust')
none_models =load_models(MAIN_DUMP,version=current_version,normalization='none')
log1p_models=load_models(MAIN_DUMP,version=current_version,normalization='log1p')
minmax_models=load_models(MAIN_DUMP,version=current_version,normalization='minmax')

In [141]:
collector = ResultsCollector()
metrics_list = []
metrics_list=[]
for norm in ['none','robust','standard','log1p','minmax']:
        current_dict=eval(f"{norm}_models")
        for model_name, ml_model in current_dict.items():
                print(f'-- Evaluating model: {model_name} {current_version} {norm} --')
                y_test, y_pred, y_proba = ml_model.y_test, ml_model.y_pred, ml_model.y_proba
                metrics = Evaluator(y_test, y_pred, y_proba).compute_metrics()
                metrics_list.append({
                        "model": ml_model.model_type,
                        "input": ml_model.dataset_name,
                        "normalization": norm,  
                        "version": current_version,
                        **metrics
                })

                collector.add(ml_model.model_type, ml_model.dataset_name, y_test, y_pred, y_proba)
results=pd.DataFrame(metrics_list)

# -- gex not related to version
results.loc[results['input']=='gene_expression', 'version'] = ''

-- Evaluating model: pytorch_mlp_Complex_protein_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: pytorch_mlp_Complex_sample_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: pytorch_mlp_concatenated_protein_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: pytorch_mlp_concatenated_sample_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: pytorch_mlp_gene_expression_gridsearch_model.joblib v2.10 none --
-- Evaluating model: pytorch_mlp_RGCN_protein_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: pytorch_mlp_RGCN_sample_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: random_forest_Complex_protein_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: random_forest_Complex_sample_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: random_forest_concatenated_protein_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model:

In [142]:
results[results['input']=='gene_expression'] # -- checking version

,model,input,normalization,version,balanced_accuracy,precision,recall,f1,mcc,auroc,auprc,brier
4,pytorch_mlp,gene_expression,none,,0.500000,0.775510,1.000000,0.873563,0.000000,0.770335,0.881820,0.182127
12,random_forest,gene_expression,none,,0.714115,0.860465,0.973684,0.913580,0.545073,0.961722,0.989846,0.092184
22,sklearn_mlp,gene_expression,none,,0.500000,0.775510,1.000000,0.873563,0.000000,0.500000,0.775510,0.227482
32,svm,gene_expression,none,,0.941388,0.973684,0.973684,0.973684,0.882775,0.976077,0.993772,0.052264
42,xgboost,gene_expression,none,,0.928230,0.972973,0.947368,0.960000,0.831005,0.964115,0.991289,0.062905


In [143]:
results_10=results

In [144]:
results.loc[results['model']=='svm']

,model,input,normalization,version,balanced_accuracy,precision,recall,f1,mcc,auroc,auprc,brier
27,svm,Complex_protein_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.944976,0.983191,0.094084
28,svm,Complex_sample_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.471292,0.772835,0.174168
29,svm,concatenated_protein_embeddings,none,v2.10,0.862440,0.968750,0.815789,0.885714,0.635411,0.935407,0.982124,0.084609
30,svm,concatenated_sample_embeddings,none,v2.10,0.627990,0.843750,0.710526,0.771429,0.224386,0.751196,0.911020,0.158547
31,svm,GAT_sample_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.842105,0.940358,0.116380
32,svm,gene_expression,none,,0.941388,0.973684,0.973684,0.973684,0.882775,0.976077,0.993772,0.052264
33,svm,GraphSAGE_sample_embeddings,none,v2.10,0.563397,0.805556,0.763158,0.783784,0.119830,0.651914,0.875326,0.187000
34,svm,RGCN_protein_embeddings,none,v2.10,0.862440,0.968750,0.815789,0.885714,0.635411,0.935407,0.982124,0.084609
35,svm,RGCN_sample_embeddings,none,v2.10,0.627990,0.843750,0.710526,0.771429,0.224386,0.751196,0.911020,0.158547
36,svm,weighted_RGCN_sample_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.447368,0.792443,0.176956


## v2.11

In [145]:
current_version='v2.11'
standard_models=load_models(MAIN_DUMP,version=current_version,normalization='standard')
robust_models=load_models(MAIN_DUMP,version=current_version,normalization='robust')
none_models =load_models(MAIN_DUMP,version=current_version,normalization='none')
log1p_models=load_models(MAIN_DUMP,version=current_version,normalization='log1p')
minmax_models=load_models(MAIN_DUMP,version=current_version,normalization='minmax')

collector = ResultsCollector()
metrics_list = []
metrics_list=[]
for norm in ['none','robust','standard','log1p','minmax']:
        current_dict=eval(f"{norm}_models")
        for model_name, ml_model in current_dict.items():
                print(f'-- Evaluating model: {model_name} {current_version} {norm} --')
                y_test, y_pred, y_proba = ml_model.y_test, ml_model.y_pred, ml_model.y_proba
                metrics = Evaluator(y_test, y_pred, y_proba).compute_metrics()
                metrics_list.append({
                        "model": ml_model.model_type,
                        "input": ml_model.dataset_name,
                        "normalization": norm,  
                        "version": current_version,
                        **metrics
                })

                collector.add(ml_model.model_type, ml_model.dataset_name, y_test, y_pred, y_proba)
results=pd.DataFrame(metrics_list)
results.loc[results['input']=='gene_expression', 'version'] = ''

-- Evaluating model: pytorch_mlp_Complex_protein_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: pytorch_mlp_Complex_sample_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: pytorch_mlp_concatenated_protein_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: pytorch_mlp_concatenated_sample_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: pytorch_mlp_RGCN_protein_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: pytorch_mlp_RGCN_sample_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: random_forest_Complex_protein_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: random_forest_Complex_sample_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: random_forest_concatenated_protein_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: random_forest_concatenated_sample_embeddings_gridsearch_model.joblib v2.11 none --
-- 

In [146]:
results_11=results

### saving csv

In [147]:
results_10.to_csv(f'../../results/score_tables/scores_v2.10_seed{seed}.csv',index=False)
results_11.to_csv(f'../../results/score_tables/scores_v2.11_seed{seed}.csv',index=False)

all_results=pd.concat([results_10,results_11],axis=0)
all_results.to_csv(f'../../results/score_tables/scores_all_seed{seed}.csv',index=False)

### analysis

In [148]:
from IPython.display import display
display(all_results)

,model,input,normalization,version,balanced_accuracy,precision,recall,f1,mcc,auroc,auprc,brier
0,pytorch_mlp,Complex_protein_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.830144,0.949142,0.174029
1,pytorch_mlp,Complex_sample_embeddings,none,v2.10,0.486842,0.770833,0.973684,0.860465,-0.077658,0.502392,0.746146,0.223446
2,pytorch_mlp,concatenated_protein_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.500000,0.775510,0.224490
3,pytorch_mlp,concatenated_sample_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.500000,0.775510,0.224490
4,pytorch_mlp,gene_expression,none,,0.500000,0.775510,1.000000,0.873563,0.000000,0.770335,0.881820,0.182127
...,...,...,...,...,...,...,...,...,...,...,...,...
157,xgboost,GraphSAGE_protein_embeddings,minmax,v2.11,0.830144,0.941176,0.842105,0.888889,0.597772,0.918660,0.975856,0.107084
158,xgboost,GraphSAGE_sample_embeddings,minmax,v2.11,0.532297,0.787234,0.973684,0.870588,0.136211,0.722488,0.883969,0.173620
159,xgboost,RGCN_protein_embeddings,minmax,v2.11,0.837321,0.923077,0.947368,0.935065,0.698439,0.858852,0.949220,0.104396
160,xgboost,weighted_RGCN_protein_embeddings,minmax,v2.11,0.830144,0.941176,0.842105,0.888889,0.597772,0.913876,0.976314,0.115985


### averaging

In [149]:
# -- take all seed files and make an average dataframe of all seeds
df0 = pd.read_csv('../../results/score_tables/scores_all_seed0.csv'); df0['seed'] = 0
df1 = pd.read_csv('../../results/score_tables/scores_all_seed1.csv'); df1['seed'] = 1
df2 = pd.read_csv('../../results/score_tables/scores_all_seed2.csv'); df2['seed'] = 2
df3 = pd.read_csv('../../results/score_tables/scores_all_seed3.csv'); df3['seed'] = 3
df4 = pd.read_csv('../../results/score_tables/scores_all_seed4.csv'); df4['seed'] = 4
df5 = pd.read_csv('../../results/score_tables/scores_all_seed5.csv'); df5['seed'] = 5
df6 = pd.read_csv('../../results/score_tables/scores_all_seed6.csv'); df6['seed'] = 6
df7 = pd.read_csv('../../results/score_tables/scores_all_seed7.csv'); df7['seed'] = 7
df8 = pd.read_csv('../../results/score_tables/scores_all_seed8.csv'); df8['seed'] = 8
df9 = pd.read_csv('../../results/score_tables/scores_all_seed9.csv'); df9['seed'] = 9


dfs = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]


# concatenate all seeds
all_df = pd.concat(dfs)

# -- those who have null version => "no version"
all_df['version'] = all_df['version'].fillna('no version').replace('', 'no version')
all_df

avg_df = all_df.groupby(['model','input','normalization','version']).mean().reset_index().drop(columns=['seed'], errors='ignore')
std_df = all_df.groupby(['model','input','normalization','version']).std().reset_index().drop(columns=['seed'], errors='ignore')

In [150]:
# -- remove the seed col
all_df['input']=='gene_expression'

0      False
1      False
2      False
3      False
4       True
       ...  
312    False
313    False
314    False
315    False
316    False
Name: input, Length: 3245, dtype: bool

In [151]:
# -- save all_df
all_df.to_csv('../../results/score_tables/scores_all_seeds.csv',index=False)
avg_df.to_csv('../../results/score_tables/scores_avg_seeds.csv',index=False)
std_df.to_csv('../../results/score_tables/scores_std_seeds.csv',index=False)

In [152]:
avg_df['input'].unique()

array(['Complex_protein_embeddings', 'Complex_sample_embeddings',
       'RGCN_protein_embeddings', 'RGCN_sample_embeddings',
       'concatenated_protein_embeddings',
       'concatenated_sample_embeddings', 'gene_expression',
       'GAT_protein_embeddings', 'GAT_sample_embeddings',
       'GraphSAGE_protein_embeddings', 'GraphSAGE_sample_embeddings',
       'weighted_RGCN_protein_embeddings',
       'weighted_RGCN_sample_embeddings'], dtype=object)